## Import all required packages

In [323]:
#importing required packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

## Declare helper functions

In [361]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[0, 255, 0], thickness=2):
    """
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, [255, 0, 0], 3)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def draw_solid_lines(img, lines, y_min, thickness=5, color=[0, 255, 0]):
    """
    Draws two solid lines (left and right), that etrapolate given line segments.
    Solid lines are drawn in color debined by 'color' rgb array, with thinkness
    value defined 'thickness', start from bottom of the image and finish at y = y_min
    
    """
    # Arrays for storing start and finish points of given segments for left and right lines
    left_segment_start_points = []
    left_segment_finish_points = []
 
    right_segment_start_points = []
    right_segment_finish_points = []
    
    min_valid_slope = 0.5
    max_valid_slope = 0.8
    
    # Parse provided lines segments and fetch theit start and finish points into proper arrays
    for line in lines:
        for x0,y0,x1,y1 in line:
            slope = (y1-y0)/(x1-x0) # calculate slope of line segment
            if slope >= 0:
                # positive slope defines the segment of right line
                if slope < min_valid_slope or slope > max_valid_slope: # Ignore all artefact lines with slope outside expected boundaries
                    continue  
                right_segment_start_points.append([x0, y0])
                right_segment_finish_points.append([x1, y1])
            else:
                # negative slope defines the segment of left line
                if slope > -min_valid_slope or slope < -max_valid_slope: # Ignore all artefact lineswith slope outside expected boundaries
                    continue 
                left_segment_start_points.append([x0, y0])
                left_segment_finish_points.append([x1, y1])
          
    y_max = img.shape[0] # overlayed detected lines should start from the bottom of the image   
    if (len(right_segment_start_points) > 0 and len(right_segment_finish_points) > 0):
        x0,y0,x1,y1 = build_line_from_segments(right_segment_start_points, right_segment_finish_points, y_min, y_max)
        # Draw calculated right line on the image
        cv2.line(img, (x0, y0), (x1, y1), color, thickness)
        
    if (len(left_segment_start_points) > 0 and len(left_segment_finish_points) > 0):
        x0,y0,x1,y1 = build_line_from_segments(left_segment_start_points, left_segment_finish_points, y_min, y_max)
        # Draw calculated left line on the image
        cv2.line(img, (x0, y0), (x1, y1), color, thickness)

def build_line_from_segments(segment_start_points, segment_finish_points, y_min, y_max):
    """
    Calculates coordinates of the line that extrapolates given line segments
    'y_min' is upped boundary by Y axle of calculated line
    'y_max' is the lower boundary by Y axle of tcalculated he line
    
    """            
    # Calculate the mean for all stored start/finish points of provided line segments
    segment_start_points_mean = np.mean(segment_start_points, axis=0)
    segment_finish_points_mean = np.mean(segment_finish_points, axis=0)
   
    # Calculate the slope value for resulting line
    slope = (segment_finish_points_mean[1] - segment_start_points_mean[1]) / (segment_finish_points_mean[0] - segment_start_points_mean[0])
    
    # Caclulate the intercept value for resulting line
    intercept = segment_finish_points_mean[1] - slope * segment_finish_points_mean[0]
     
    # Calculate coordinates for resulting line
    y0 = y_min
    x0 = int((y0 - intercept) / slope)
    
    y1 = y_max  
    x1 = int((y1 - intercept) / slope)
    
    return x0,y0,x1,y1

def hough_solid_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    Returns the given image with drawn left and right solid lines.
    Drawn lines are calculated using hough transformation
    and further extrapolation of calculated line segments.
    
    `img` should be the output of a Canny Edge transformation.

    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    # top of drawn lines should be limited by a horizont line that is bit lower than middle of the picture height
    draw_solid_lines(line_img, lines, int(line_img.shape[0]/2) + 80) 
    return line_img    

def augument_image_with_lane_lines(img):
    """
    This function represent the pipeline for detecting lines on the road lane displayed in given image 'img'
    
    `img` is the original image to be processed
   
    Returns the original image 'img' with detected lane lines overlayed
    """
    # define params for algorythm tuning
    blur_kernel_size = 3
    canny_low_threshold = 50
    canny_high_threshold = 217

    hough_tf_rho = 2 # distance resolution in pixels of the Hough grid
    hough_tf_theta = np.pi/180 # angular resolution in radians of the Hough grid
    hough_tf_threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    hough_tf_min_line_length = 40 #minimum number of pixels making up a line
    hough_tf_max_line_gap = 20   # maximum gap in pixels between connectable line segments
    
    img_height = img.shape[0]
    img_width = img.shape[1]

    # Get greyscalled copy of original image
    grayscale_image = grayscale(img)
  
    # apply Gaussian smoothing to grayscale image
    blured_grayscale_image = gaussian_blur(grayscale_image, blur_kernel_size)

    # apply Canny edge detection on blured grayscale image
    canny_edges_image = canny(blured_grayscale_image, canny_low_threshold, canny_high_threshold)

    # define vertices of 'region of interest' polygon
    polygon_vertices = np.array([[(100, img_height),(img_width - 100, img_height), (img_width/2 + 70, img_height/2 + 70), (img_width/2 - 70, img_height/2 + 70)]], dtype=np.int32)

    # create a masked edges image (mask out everything excpet the region of interest)
    masked_canny_edges_image = region_of_interest(canny_edges_image, polygon_vertices)

    # apply Hough transformation to masked edge detected image to get image with detected lines drawn (additionally extrapolated)
    lines = hough_solid_lines(masked_canny_edges_image, hough_tf_rho, hough_tf_theta, hough_tf_threshold, hough_tf_min_line_length, hough_tf_max_line_gap)

    # draw detected lines on top of original image (by overlaying two images)
    image_with_lines = weighted_img(lines, img)
    return image_with_lines

def process_images(img_input_dir, img_output_dir):
    """
    Processes all images in the directory 'img_input_dir'
    and stores the resulting images in 'img_output_dir' directory
    with the same file names as source image files
    
    """

    for file_name in os.listdir(img_input_dir):
        # Skip all files not having .jpg extension.
        if not file_name.endswith(".jpg"):
            continue
        
        # read in the original image
        original_image = mpimg.imread(img_input_dir + file_name)
        # process image
        augumented_image = augument_image_with_lane_lines(original_image)
        # artificially convert colors rgb to bgr to make cv2 to store the image in rgb
        image_with_lines_bgr = cv2.cvtColor(augumented_image, cv2.COLOR_RGB2BGR)
        
        # save processed image to file with original name into output directory
        cv2.imwrite(img_output_dir + file_name, image_with_lines_bgr)

## Test the lane finding pipeline on bunch of test images

In [362]:
# call a processing pipeline for all images in "test_images/" directory
# and store processed images in "test_images_output/" with same name 
process_images("test_images/", "test_images_output/")

## Test lane finding pipeline on several video files

In [329]:
# test pipeline on video file solidWhiteRight.mp4 and store resulting video into 'test_videos_output'directory
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(augument_image_with_lane_lines) 
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:06<00:00, 35.75it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 2.29 s, sys: 300 ms, total: 2.59 s
Wall time: 7.06 s


In [328]:
# test the pipeline on video file solidYellowLeft.mp4 and store resulting video into 'test_videos_output' directory
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(augument_image_with_lane_lines)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:21<00:00, 31.20it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 7.51 s, sys: 1 s, total: 8.51 s
Wall time: 22.8 s
